In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
import tensorflow as tf

%matplotlib notebook

from IPython.display import display, HTML

pd.set_option('display.max_columns', None)

from pandas.api.types import CategoricalDtype
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [4]:
def RNNConfig(object):
    def __init__(
        cell_type='lstm', window=20, forget_bias=1.0, 
        n_hidden_cells=(100), keep_prob=1.0, batch_size=64, epoch_num=100,
        learning_rate=0.01, max_grad_norm=1.0, init_scale=0.1,
    ):
        self.cell_type = cell_type
        self.window = window
        self.forget_bias = forget_bias
        self.n_hidden_cells = n_hidden_cells
        self.keep_prob = keep_prob
        self.batch_size = batch_size
        self.epoch_num = epoch_num
        self.learning_rate = learning_rate
        self.max_grad_norm = max_grad_norm
        self.init_scale = init_scale

In [3]:
data = pd.read_csv(
    "../data/raw/train_values.csv", 
    index_col='row_id', 
    parse_dates=['timestamp'], 
    nrows=100000  
    # We'll just load the first 100K so we know
    # how to load the data using code
)

# Data preprocessing

## Encoding data

Let's figure out how to preprocess the data into RNN-feedable form.

* `phase` will NOT be one-hot-encoded since there is an order to it, so we can simply encode it using 1, 2, .., 5.

In [5]:
phase_categorical = CategoricalDtype(
    categories=['pre_rinse', 'caustic', 'intermediate_rinse', 
                'acid', 'final_rinse'],
    ordered=True
)
data.phase = data.phase.astype(phase_categorical)

data.phase, phase_mapping_idx = data.phase.factorize()

* Ensure each sequence is in the right order using the `timestamp` column, but once sequences are set up, discard the column.
  * The column is only useful insofar as it tells us which data point comes before or after others.
  * But then again, **perhaps there is some signal from absoluate passage of time, so we can consider encoding this into UNIX times.** 
 Something to try out.  

In [6]:
data.timestamp = data.timestamp.view(int)

* Ensure each sequence is from single process using `process_id`, but don't include it in data since it really is just an ID column.

* One-hot encode `object_id` and `pipeline`.
  * Although `object_id` is an ID column, there aren't that many objects (~100? **NOT SURE**) and there are multiple processes over each object. So `object_id` may carry valuable information.
  * I'm not sure what `pipeline` is, but we'll treat it as a categorical column

In [16]:
categorical_features = ['object_id', 'pipeline',]

In [41]:
from sklearn.preprocessing import LabelEncoder


class DataframeLabelEncoder(object):
    def __init__(self, categorical_features):
        assert isinstance(categorical_features, (list, np.ndarray))
        self.categorical_features = categorical_features
        self.feature_encoder = dict()
        self.cat_feature_mask = None
        
    def fit(self, dataframe):
        assert isinstance(dataframe, pd.DataFrame)
        
        if self.cat_feature_mask is None:
            self.cat_feature_mask = np.zeros(shape=dataframe.shape[1], dtype=np.int16)
        
        for i, feature in enumerate(categorical_features):
            le = LabelEncoder()
            le.fit(dataframe[feature])
            self.feature_encoder[feature] = le
            
            self.cat_feature_mask[i] = 1
        
    def transform(self, dataframe):
        assert dataframe.shape[1] == self.cat_feature_mask.shape[-1]
        
        array = dataframe.values.copy()
        for feature in categorical_features:
            encoded = self.feature_encoder[feature].transform(
                dataframe[feature]
            )
            index = np.where(dataframe.columns == feature)[0][0]
            array[:, index] = encoded
            
        return array
    
    def fit_transform(self, dataframe):
        self.fit(dataframe)
        return self.transform(dataframe)

In [42]:
label_encoder = DataframeLabelEncoder(categorical_features)
label_encoder.fit(data)
encoded_data = label_encoder.transform(data)

In [44]:
# Create One-hot encoder
ohe = OneHotEncoder(categorical_features=label_encoder.cat_feature_mask)
ohe.fit(data)

OneHotEncoder(categorical_features=array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int16),
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values='auto', sparse=True)

In [47]:
a = ohe.transform(data)

In [48]:
data.shape

(100000, 35)

In [50]:
a.toarray().shape

(100000, 158)

In [51]:
len(data['object_id'].unique()) + len(data['pipeline'].unique())

52

Note that we've only fitted the one-hot encoder. Using `transform` returns a numpy output. Will deal with this later.

## Scaling data

From EDA, it looks like some are normally distributed, some exponential... we'll just standardize.

In [14]:
data['tank_lsh_acid'] = data['tank_lsh_acid'].astype(bool)
data['tank_lsh_pre_rinse'] = data['tank_lsh_pre_rinse'].astype(bool)
scaling_cols = data.select_dtypes([np.float, np.int]).columns
scaler = StandardScaler()
scaler.fit(data[scaling_cols])

StandardScaler(copy=True, with_mean=True, with_std=True)

Let's use `tf.Example`

In [9]:
labels = pd.read_csv("../data/raw/train_labels.csv")

Ok let's just create the padded array now. (What Holden did!) 

From our EDA, we know the maximum length of a process is **15107**.
We also know there are 

In [52]:
for pid in data.process_id.unique():
    pid_mask = data.process_id == pid
    turbidity = labels[pid]
    process_data = ohe.transform(data[pid_mask])
    numerical_data = scaler.transform(process_data[scaling_cols])
    nrows = data[pid_mask].shape[0]
    np.zeros()

KeyError: 20001

## Reshaping data

But first, we need to figure out how we're going to approach this problem.

Key things to consider:

* **Sequence length**: the processes have different lengths
  * Not only that, the longest process has 15107 data points---very long!!
* **Prediction output**: for each process (=sequence), predict **one** value.

### Turning variable-length sequences into fixed-length?
Say our lookback window size is $L$. We want to reshape data so that each process is expressed in $n$ sequences of length $L$ where $n= \text{length of process} - L + 1$. 

At the end of each process, i.e. at $n$th sequence, the RNN's state must be refreshed.

It is probably the best to create an iterator rather than making and actual array of all of this. Some iterator that managers each process and lookback length as well as signal to refresh RNN states.

Following suggestions from https://danijar.com/variable-sequence-lengths-in-tensorflow/

In [ ]:
def length(sequence):
  used = tf.sign(tf.reduce_max(tf.abs(sequence), 2))
  length = tf.reduce_sum(used, 1)
  length = tf.cast(length, tf.int32)
  return length

In [ ]:
def batch_iterator(dataX, dataY, batch_size, num_steps):
    data_len = len(dataY)
    batch_len = int(data_len / batch_size)

    if batch_len == 0:
        raise ValueError("epoch_size == 0, decrease batch_size or num_steps")

    for i in range(batch_len):
        input_x = dataX[i * batch_size: (i + 1) * batch_size]
        input_y = dataY[i * batch_size: (i + 1) * batch_size]

        yield (input_x, input_y)

In [ ]:
def many_to_one_model_reproduce(output_dim=1):
    model = tf.keras.models.Sequential(
        layers=[
            tf.keras.layers.LSTM(
                rnn_cell_hidden_dim, input_shape=[14, 17], return_sequences=True,
                activation='softsign'),
            tf.keras.layers.LSTM(
                rnn_cell_hidden_dim, activation='softsign'),
            tf.keras.layers.Dense(
                output_dim,
                activation='softmax' if output_dim == 2 else 'sigmoid'),
        ]
    )
    loss = 'binary_crossentropy' if output_dim == 1 else 'categorical_crossentropy'
    adam = tf.keras.optimizers.Adam(lr=learning_rate)
    model.compile(
        loss=loss, optimizer=adam,
        metrics=['accuracy', precision, recall, f1_score]
    )

    return model